In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mtcnn

In [ ]:
import tensorflow as tf
from mtcnn import mtcnn
import PIL
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# https://jkisaaclee.kro.kr/keras/facenet/deep%20learning/computer%20vision/2019/10/01/how_to_develop_a_face_recognition_system_using_facenet_in_keras_ko/
# https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

In [ ]:
print(model.input)
print(model.output)

In [ ]:
!pip install mtcnn
!pip install mediapipe

In [ ]:
friends_path_first = '/content/drive/MyDrive/A.아이/images/friends_image/fist_image/'
friends_path_first_name = os.listdir(friends_path_first)

friends_path_second = '/content/drive/MyDrive/A.아이/images/friends_image/second_image/'
friends_path_second_name = os.listdir(friends_path_second)

In [ ]:
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
image_friends_first_list = []
margin = 0.15  # Adjust this value to change the cropping margin

for name in friends_path_first_name :
    img = cv2.imread(friends_path_first + name)
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
            face = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if face.detections:
                for i, detection in enumerate(face.detections):
                    # Get bounding box coordinates
                    box = detection.location_data.relative_bounding_box
                    ih, iw, _ = img.shape
                    x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

                    # Calculate margins for x, y, w, h
                    mx = int(w * margin)
                    my = int(h * margin)

                    # Crop the face with margins
                    face_image = img[y+my:y+h-my, x+mx:x+w-mx]
                    if face_image.size != 0:
                        # Resize the image to 105x105
                        face_image = cv2.resize(face_image, (160, 160))
                        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                    else :
                        break
            else :
                print('cant find face')
                continue
    image_friends_first_list.append(face_image)

for i in image_friends_first_list :
    cv2_imshow(i)


In [ ]:
mp_face_detection = mp.solutions.face_detection
image_friends_second_list = []
margin = 0.15  # Adjust this value to change the cropping margin

for name in friends_path_second_name :
    img = cv2.imread(friends_path_second + name)
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
            face = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if face.detections:
                for i, detection in enumerate(face.detections):
                    # Get bounding box coordinates
                    box = detection.location_data.relative_bounding_box
                    ih, iw, _ = img.shape
                    x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

                    # Calculate margins for x, y, w, h
                    mx = int(w * margin)
                    my = int(h * margin)

                    # Crop the face with margins
                    face_image = img[y+my:y+h-my, x+mx:x+w-mx]
                    if face_image.size != 0:
                        # Resize the image to 105x105
                        face_image = cv2.resize(face_image, (160, 160))
                        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                    else :
                        break
            else :
                print('cant find face')
                continue
    image_friends_second_list.append(face_image)

for count,i in enumerate(image_friends_second_list) :
    print(count)
    cv2_imshow(i)


In [ ]:
image_friends_first_list = np.array(image_friends_first_list)
image_friends_second_list = np.array(image_friends_second_list)

image_friends_first_list.shape, image_friends_second_list.shape

In [ ]:
face_embeding_model = tf.keras.models.load_model('/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

In [ ]:
import matplotlib.pyplot as plt

def get_face_embedding(image, model):
    # Preprocess the image
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image.astype('float32') / 255  # Normalize to [0,1]

    # Compute embedding
    embedding = model.predict(image)[0]
    return embedding

def test_model_with_friends(index, image_friends_first_list, image_friends_second_list, model):
    # Get the embedding for the anchor image
    anchor_embedding = get_face_embedding(image_friends_first_list[index], model)

    # Variables to keep track of most similar images
    most_similar_images = []  # A list to keep track of the indices of the most similar images
    distances = []  # A list to keep track of the distances

    # Iterate over images to compute distances
    for i, image in enumerate(image_friends_second_list):
        # Get the embedding for this image
        image_embedding = get_face_embedding(image, model)

        # Compute the distance
        distance = np.linalg.norm(anchor_embedding - image_embedding)
        distances.append(distance)

    # Compute min and max distances
    min_distance = min(distances)
    max_distance = max(distances)

    # Normalize distances and compute similarities
    for i, distance in enumerate(distances):
        # Normalize the distance to [0,1] using the min and max distances
        normalized_distance = (distance - 0) / (max_distance - 0)

        # Convert to percentage
        percentage = (1 - normalized_distance) * 100

        # Compute the similarity (as 1 - normalized Euclidean distance)
        similarity = percentage

        # Add new image
        most_similar_images.append((i, similarity))  # Keeping track of the index and similarity

    # Sort the images by similarity
    most_similar_images.sort(key=lambda x: x[1], reverse=True)

    # Show the anchor image
    plt.figure(figsize=(20, 4))
    plt.subplot(1, 6, 1)
    plt.imshow(image_friends_first_list[index])
    plt.title("Anchor image")
    plt.axis('off')

    # Show the most similar images
    for i, (img_index, similarity) in enumerate(most_similar_images[:5]):
        plt.subplot(1, 6, i+2)
        plt.imshow(image_friends_second_list[img_index])
        plt.title(f'Similarity: {similarity:.5f}%')
        plt.axis('off')
    plt.show()

In [ ]:
test_model_with_friends(4, image_friends_first_list, image_friends_second_list, face_embeding_model)

In [ ]:
from mtcnn import mtcnn
import tensorflow as tf
import mediapipe as mp
import PIL
import numpy as np
import os
import cv2

class get_embeding_vector() :
    def __init__(self) :
        self.mp_face_detection = mp.solutions.face_detection
        self.margin = 0.15  # Adjust this value to change the cropping margin
        self.model = tf.keras.models.load_model('/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

    def get_face(self, image) :
        img = cv2.imread(image)
        with self.mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
                face = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                if face.detections:
                    for detection in face.detections :
                        # Get bounding box coordinates
                        box = detection.location_data.relative_bounding_box
                        ih, iw, _ = img.shape
                        x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

                        # Calculate margins for x, y, w, h
                        mx = int(w * self.margin)
                        my = int(h * self.margin)

                        # Crop the face with margins
                        face_image = img[y+my:y+h-my, x+mx:x+w-mx]
                        if face_image.size != 0:
                            # Resize the image to 105x105
                            face_image = cv2.resize(face_image, (160, 160))
                            face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                        else :
                            print('face image size is null')
                            break
                else :
                    print('cant find face')
        return face_image

    def get_embeded_face(self, face_image) :
        # Preprocess the image
        image = cv2.cvtColor(face_image, cv2.COLOR_GRAY2RGB)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image = image.astype('float32') / 255  # Normalize to [0,1]

        # Compute embedding
        embedding = self.model.predict(image)[0]

        return embedding

In [ ]:
face_embeding = get_embeding_vector()

In [ ]:
face_embeding.get_embeded_face(image_friends_first_list[0])

In [ ]:
from mtcnn import MTCNN
import tensorflow as tf
import mediapipe as mp
import PIL
import numpy as np
import os
import cv2
from typing import Optional

class FaceEmbedder:
    def __init__(self, model_path: str, margin: float = 0.15) :
        self.margin = margin  # Adjust this value to change the cropping margin
        self.model = tf.keras.models.load_model(model_path)
        self.face_detection = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

    def _get_face(self, img) -> Optional[np.ndarray]:
        img = cv2.imread(img)
        face = self.face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not face.detections:
            print('Cant find face')
            return None

        for detection in face.detections:
            # Get bounding box coordinates
            box = detection.location_data.relative_bounding_box
            ih, iw, _ = img.shape
            x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

            # Calculate margins for x, y, w, h
            mx = int(w * self.margin)
            my = int(h * self.margin)

            # Crop the face with margins
            face_image = img[y+my:y+h-my, x+mx:x+w-mx]
            if face_image.size == 0:
                print('Face image size is null')
                continue

            # Resize the image to 105x105
            return cv2.resize(face_image, (160, 160))

        return None

    def get_embedded_face(self, face_image) -> Optional[np.ndarray]:
        img = cv2.cvtColor(face_image, cv2.COLOR_GRAY2RGB)
        if img is None:
            print(f"Failed to read image: {face_image}")
            return None

        face_image = self._get_face(img)
        if face_image is None:
            return None

        # Preprocess the image
        image = np.expand_dims(face_image, axis=0)  # Add batch dimension
        image = image.astype('float32') / 255  # Normalize to [0,1]

        # Compute embedding
        return self.model.predict(image)[0]


In [ ]:
embeding = FaceEmbedder(model_path = '/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

In [ ]:
embeding.get_embedded_face(image_friends_first_list[0])